# **Bioinformatics Project**
---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2.5 million compounds. It is compiled from more than 92,100 documents, 1.7 million assays and the data spans 16,000 targets and 2,100 cells and 48,800 indications.
[Data of May 07, 2025].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

## **Importing libraries**

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

### **Target search for MMP-13**

In [ ]:
# Target search for MMP-13
target = new_client.target
target_query = target.search('CHEMBL280')
targets = pd.DataFrame.from_dict(target_query)
targets

:### **Select and retrieve bioactivity data for *MMP13* (First Entry)**

---



We will assign the 1st entry (which corresponds to the target protein, *MMP13*) to the ***selected_target*** variable

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

Here, we will retrieve only bioactivity data for *MMP13* (CHEMBL280) that are reported as pChEMBL values.

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df

## **Handling missing data**
If any compounds has missing value for the **standard_value** and **canonical_smiles** column then drop it.

In [ ]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

In [ ]:
len(df2.canonical_smiles.unique())

In [ ]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

## **Data pre-processing of the bioactivity data**

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

Saves dataframe to CSV file

In [ ]:
df3.to_csv('mmp13-2665.csv', index=False)

In [ ]:
from google.colab import files
files.download('mmp13-2665.csv')

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1nM and 100 nM will be considered to be **active** while those greater than 10,00 nM will be considered to be **inactive**.

In [ ]:
import pandas as pd

In [ ]:
df4 = pd.read_csv('mmp13-2665.csv')

In [ ]:
df4

In [ ]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 100:
    bioactivity_threshold.append("inactive")
  else: float(i) <= 100:
    bioactivity_threshold.append("active")

In [ ]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

Saves dataframe to CSV file

In [ ]:
df5.to_csv('mmp13.csv', index=False)

In [ ]:
from google.colab import files
files.download('mmp13.csv')

# **Exploratory Data Analysis**
---

## **Install conda and rdkit**

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

## **Load bioactivity data**

In [ ]:
df5_no_smiles = df5.drop(columns='canonical_smiles')

In [ ]:
df5_no_smiles

In [ ]:
smiles = []

for i in df5.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')

In [ ]:
df5_clean_smiles = pd.concat([df5_no_smiles,smiles], axis=1)
df5_clean_smiles

In [ ]:
df1 = df5_clean_smiles.dropna()

In [ ]:
df1


## **Calculate Lipinski descriptors**
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the **druglikeness** of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the **Rule-of-Five** or **Lipinski's Rule**.

The Lipinski's Rule stated the following:
* Molecular weight < 500 Dalton
* Octanol-water partition coefficient (LogP) < 5
* Hydrogen bond donors < 5
* Hydrogen bond acceptors < 10

### **Import libraries**

In [ ]:
!pip install rdkit-pypi


In [ ]:
from rdkit.Chem import Descriptors, Lipinski


### **Calculate descriptors**

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import numpy as np
import pandas as pd

def lipinski(smiles, verbose=False):
    moldata = []
    for elem in smiles:
        mol = Chem.MolFromSmiles(elem)
        moldata.append(mol)

    baseData = np.arange(1, 1)
    i = 0
    for mol in moldata:
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
        desc_NumRotatableBonds = Lipinski.NumRotatableBonds(mol)


        # Calculate TPSA
        desc_TPSA = Descriptors.TPSA(mol)

        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors,
                        desc_NumRotatableBonds,
                        desc_TPSA  # Add TPSA to the array
                        ])

        if i == 0:
            baseData = row
        else:
            baseData = np.vstack([baseData, row])
        i += 1

    columnNames = ["MW", "LogP", "NumHDonors", "NumHAcceptors", "NumRotatableBonds", "TPSA"]
    descriptors = pd.DataFrame(data=baseData, columns=columnNames)

    return descriptors

In [ ]:
df_lipinski = lipinski(df1.canonical_smiles)
df_lipinski

### **Combine DataFrames**

Let's take a look at the 2 DataFrames that will be combined.

In [ ]:
df_lipinski

In [ ]:
df6 = df1

Now, let's combine the 2 DataFrame

In [ ]:
df_combined = pd.concat([df6,df_lipinski], axis=1)

In [ ]:
df_combined

In [ ]:
df_combined.to_csv('Lipinski.csv')

In [ ]:
from google.colab import files
files.download('Lipinski.csv')

### **Convert IC50 to pIC50**
To allow **IC50** data to be more uniformly distributed, we will convert **IC50** to the negative logarithmic scale which is essentially **-log10(IC50)**.

This custom function pIC50() will accept a DataFrame as input and will:
* Take the IC50 values from the ``standard_value`` column and converts it from nM to M by multiplying the value by 10$^{-9}$
* Take the molar value and apply -log10
* Delete the ``standard_value`` column and create a new ``pIC50`` column

In [ ]:
# https://github.com/chaninlab/estrogen-receptor-alpha-qsar/blob/master/02_ER_alpha_RO5.ipynb

import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', axis=1)

    return x


Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.

In [ ]:
df_combined.standard_value.describe()

In [ ]:
-np.log10( (10**-9)* 100000000 )

In [ ]:
-np.log10( (10**-9)* 10000000000 )

We will first apply the norm_value() function so that the values in the standard_value column is normalized.

In [ ]:

def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', axis=1)

    return x

In [ ]:
df_norm = norm_value(df_combined)
df_norm

In [ ]:
df_norm.standard_value_norm.describe()

In [ ]:
df_final = pIC50(df_norm)
df_final

In [ ]:
df_final.to_csv('MMP13_pIC50.csv')

In [ ]:
from google.colab import files
files.download('MMP13_pIC50.csv')

In [ ]:
df_final.pIC50.describe()

## **Exploratory Data Analysis (Chemical Space Analysis) via Lipinski descriptors**

### **Import library**

In [ ]:
df_final=pd.read_csv('MMP13_pIC50_cleaned.csv')

In [ ]:
df_final

In [ ]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

### **Frequency plot of the 2 bioactivity classes**

In [ ]:
plt.figure(figsize=(6.5, 6.5))
order = ['active','inactive']

# Set the order of the bars

sns.countplot(x='bioactivity_class', data=df_final, edgecolor='black', order=order, hue='bioactivity_class')

plt.xlabel('bioactivity_class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('plot_bioactivity_class.pdf')

### **Scatter plot of MW versus LogP**

It can be seen that the 2 bioactivity classes are spanning similar chemical spaces as evident by the scatter plot of MW vs LogP.

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.scatterplot(x='MW', y='LogP', data=df_final, hue='bioactivity_class', size='pIC50', edgecolor='black', alpha=0.5)

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('plot_MW_vs_LogP.pdf')

### **Box plots**

#### **pIC50 value**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'pIC50', data = df_final, hue='bioactivity_class')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('plot_ic50.pdf')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
def mannwhitney(descriptor, verbose=False):
  # https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/
  from numpy.random import seed
  from numpy.random import randn
  from scipy.stats import mannwhitneyu

# seed the random number generator
  seed(1)

# actives and inactives
  selection = [descriptor, 'bioactivity_class']
  df = df_final[selection]
  active = df[df.bioactivity_class == 'active']
  active = active[descriptor]

  selection = [descriptor, 'bioactivity_class']
  df = df_final[selection]
  inactive = df[df.bioactivity_class == 'inactive']
  inactive = inactive[descriptor]

# compare samples
  stat, p = mannwhitneyu(active, inactive)
  #print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
  alpha = 0.05
  if p > alpha:
    interpretation = 'Same distribution (fail to reject H0)'
  else:
    interpretation = 'Different distribution (reject H0)'

  results = pd.DataFrame({'Descriptor':descriptor,
                          'Statistics':stat,
                          'p':p,
                          'alpha':alpha,
                          'Interpretation':interpretation}, index=[0])
  filename = 'mannwhitneyu_' + descriptor + '.csv'
  results.to_csv(filename)

  return results

In [ ]:
mannwhitney('pIC50')

#### **MW**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'MW', data = df_final, hue='bioactivity_class')

plt.xlabel('MW Comparison', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('plot_MW.pdf')

In [ ]:
mannwhitney('MW')

#### **LogP**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'LogP', data = df_final, hue='bioactivity_class')

plt.xlabel('LogP Comparison', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('plot_LogP.pdf')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
mannwhitney('LogP')

#### **NumHDonors**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHDonors', data = df_final, hue='bioactivity_class')

plt.xlabel('nHD Comparison', fontsize=14, fontweight='bold')
plt.ylabel('nHD', fontsize=14, fontweight='bold')

plt.savefig('plot_NumHDonors.pdf')

**Statistical analysis | Mann-Whitney U Test**

In [ ]:
mannwhitney('NumHDonors')

#### **NumHAcceptors**

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumHAcceptors', data = df_final, hue='bioactivity_class')

plt.xlabel('nHA Comparison', fontsize=14, fontweight='bold')
plt.ylabel('nHA', fontsize=14, fontweight='bold')


plt.savefig('plot_NumHAcceptors.pdf')

In [ ]:
mannwhitney('NumHAcceptors')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'TPSA', data = df_final, hue='bioactivity_class')

plt.xlabel('TPSA Comparison', fontsize=14, fontweight='bold')
plt.ylabel('TPSA', fontsize=14, fontweight='bold')

plt.savefig('plot_TPSA.pdf')

In [ ]:
mannwhitney('TPSA')

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'bioactivity_class', y = 'NumRotatableBonds', data = df_final, hue='bioactivity_class')

plt.xlabel('nRot Comparison', fontsize=14, fontweight='bold')
plt.ylabel('nRot', fontsize=14, fontweight='bold')

plt.savefig('plot_TPSA.pdf')

In [ ]:
mannwhitney('NumRotatableBonds')

# **BDescriptor Calculation and Dataset Preparation**
---

## **Download PaDEL-Descriptor**

In [ ]:
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.zip
! wget https://github.com/dataprofessor/bioinformatics/raw/master/padel.sh

In [ ]:
! unzip padel.zip

In [ ]:
df_final

In [ ]:
df3 = df_final

In [ ]:
selection = ['canonical_smiles','molecule_chembl_id']
df3_selection = df3[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
! cat molecule.smi | head -5

In [ ]:
! cat molecule.smi | wc -l

## **Calculate fingerprint descriptors**


### **Calculate PaDEL descriptors**

In [ ]:
! cat padel.sh

In [ ]:
! bash padel.sh

In [ ]:
! ls -l

In [ ]:
df3_X = pd.read_csv('descriptors_output.csv')

In [ ]:
from google.colab import files
files.download('descriptors_output.csv')

## **Preparing the X and Y Data Matrices**

### **X data matrix**

In [ ]:
import pandas as pd

In [ ]:
df3_X=pd.read_csv('/content/descriptors_output_pIC50.csv')

In [ ]:
df3_X

In [ ]:
df_7 = df3_X.drop(columns=['molecule_chembl_id', 'pIC50', 'bioactivity_class', 'canonical_smiles', 'MW', 'LogP', 'NumHDonors', 'NumHAcceptors', 'NumRotatableBonds', 'TPSA'])
df_7

In [ ]:
import pandas as pd
if not isinstance(df_7, pd.DataFrame):
    df_7 = pd.DataFrame(df_7)

# Calculate correlation matrix
correlation_matrix = df_7.corr()

# Find columns with high correlation
high_corr_columns = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.90:
            colname = correlation_matrix.columns[i]
            high_corr_columns.add(colname)

# Drop columns with high correlation
df_7_filtered = df_7.drop(columns=high_corr_columns)
print(f"Removed columns: {high_corr_columns}")

### **3.4. Remove low variance features**

In [ ]:
from sklearn.feature_selection import VarianceThreshold
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))
df_7_filtered = selection.fit_transform(df_7_filtered)

In [ ]:
X=df_7_filtered

In [ ]:
X

In [ ]:
Y=df3_X['bioactivity_class']

In [ ]:
Y=df3_X['pIC50']

## **Combining X and Y variable**

## **1. Import libraries**

### **3.3. Let's examine the data dimension**

## **4. Data split (80/20 ratio)**

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split


In [ ]:
from sklearn.utils import shuffle

X, Y = shuffle(X, Y, random_state=42)  # Shuffle X and Y together

# ... then continue with train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
X_test.shape, Y_test.shape

# **Model Building**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb

In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

# Initialize GradientBoostingClassifier and GridSearchCV
gbc = GradientBoostingClassifier(random_state=42)
grid_search = GridSearchCV(gbc, param_grid=param_grid, cv=10, scoring='accuracy', verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_gbc = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

In [ ]:
!pip install lightgbm
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# Define the hyperparameter grid
param_grid = {
    'max_depth': [3, 5, 9],
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [300, 500, 700]
}

# Initialize LGBMClassifier and GridSearchCV
lgbm = lgb.LGBMClassifier(random_state=42)
grid_search = GridSearchCV(lgbm, param_grid=param_grid, cv=10, scoring='accuracy', verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_lgbm = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Make predictions on the test set
Y_pred = best_lgbm.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy on test set: {:.2f}".format(accuracy))

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Test different numbers of neighbors
    'weights': ['uniform', 'distance'],  # Test different weight functions
    'metric': ['euclidean', 'manhattan']  # Test different distance metrics
}

# Initialize KNeighborsClassifier and GridSearchCV
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid=param_grid, cv=10, scoring='accuracy', verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_knn = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Make predictions on the test set
Y_pred = best_knn.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy on test set: {:.2f}".format(accuracy))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 300, 500, 700, 900],
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [3,5,7]
}

# Initialize ExtraTreesClassifier and GridSearchCV
etc = ExtraTreesClassifier(random_state=42)
grid_search = GridSearchCV(etc, param_grid=param_grid, cv=10, scoring='accuracy', verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_etc = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Make predictions on the test set
Y_pred = best_etc.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy on test set: {:.2f}".format(accuracy))

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [3,5,7]
}

# Initialize DecisionTreeClassifier and GridSearchCV
dtc = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dtc, param_grid=param_grid, cv=10, scoring='accuracy', verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_dtc = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Make predictions on the test set
Y_pred = best_dtc.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy on test set: {:.2f}".format(accuracy))

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np #Added this line to import the numpy module
np.random.seed(42)

# Define the hyperparameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-4, 4, 20),
    'solver': ['liblinear', 'sag', 'saga', 'newton-cg', 'lbfgs'],
    'max_iter':[1000]
}

# Initialize LogisticRegression and GridSearchCV
logreg = LogisticRegression(random_state=42, max_iter=1000)
grid_search = GridSearchCV(logreg, param_grid=param_grid, cv=10, scoring='accuracy', verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_logreg = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Make predictions on the test set
Y_pred = best_logreg.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy on test set: {:.2f}".format(accuracy))

In [ ]:
param_grid = {'C': [0.1, 1, 10],
              'gamma': [0.1, 1, 10]}
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

grid_search = GridSearchCV(SVC(kernel='rbf', random_state=42),
                           param_grid, cv=10)
grid_search.fit(X_train, Y_train)
print("Best parameters found: ", grid_search.best_params_)
best_svm = grid_search.best_estimator_
# Make predictions on the test set
Y_pred = best_svm.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy on test set: {:.2f}".format(accuracy))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the hyperparameter grid for GaussianProcessClassifier
param_grid = {
    'kernel': [RBF(length_scale=length_scale)
               for length_scale in [0.1, 1.0, 10.0]],
    'kernel__length_scale_bounds': [(1e-5, 1e3)]
}

# Initialize GaussianProcessClassifier and GridSearchCV
gpc = GaussianProcessClassifier(random_state=42)
grid_search = GridSearchCV(gpc, param_grid, cv=10)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_gpc = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)
# Make predictions on the test set
Y_pred = best_gpc.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy on test set: {:.2f}".format(accuracy))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the hyperparameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [100, 300, 500,700],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [3,5,7],
    'criterion': ['gini', 'entropy']
}

# Initialize RandomForestClassifier and GridSearchCV
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_rf = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)

# Make predictions on the test set
Y_pred = best_rf.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy on test set: {:.2f}".format(accuracy))

In [ ]:
names = ["Nearest_Neighbors", "SVC", "Gaussian_Process","Gradient_Boosting", "DecisionTreeClassifier", "ExtraTreesClassifier", "RandomForestClassifier", "LogisticRegression", "lightgbm"]

from sklearn.gaussian_process.kernels import RBF
classifiers = [
    KNeighborsClassifier(n_neighbors=11, metric='manhattan', weights= 'distance'),
    SVC(kernel="rbf", C=1, gamma=0.1, random_state=42),
    GaussianProcessClassifier(1.0 * RBF(length_scale=1, length_scale_bounds=(1e-05, 1000.0)), random_state=42),
    GradientBoostingClassifier(n_estimators=500 , max_depth=3, min_samples_split=2, learning_rate=0.1, random_state=42),
    DecisionTreeClassifier(criterion= 'entropy', max_depth= 15, max_features= 5, min_samples_leaf= 1, min_samples_split= 2),
    ExtraTreesClassifier(criterion= 'gini', max_depth= 15, max_features= 7, min_samples_leaf= 2, min_samples_split= 10, n_estimators= 100),
    RandomForestClassifier(criterion= 'entropy', max_depth= 15, max_features= 7, min_samples_leaf= 1, min_samples_split= 10, n_estimators= 500),
    LogisticRegression(C=0.23357214690901212, max_iter=1000, penalty= 'l2', solver= 'lbfgs'),
    lgb.LGBMClassifier(n_estimators=700, learning_rate=0.01, max_depth=5, random_state=42)]

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef
from sklearn.model_selection import cross_val_predict
import pandas as pd

# Evaluate each classifier on Train, CV, and Test sets
results = []

for name, clf in zip(names, classifiers):
    # Training set evaluation
    clf.fit(X_train, Y_train)
    Y_train_pred = clf.predict(X_train)

    accuracy_train = accuracy_score(Y_train, Y_train_pred)

    mcc_train = matthews_corrcoef(Y_train, Y_train_pred)

    # Cross-validation
    Y_cv_pred = cross_val_predict(clf, X_train, Y_train, cv=10)

    accuracy_cv = accuracy_score(Y_train, Y_cv_pred)

    mcc_cv = matthews_corrcoef(Y_train, Y_cv_pred)

    # Test set evaluation
    Y_test_pred = clf.predict(X_test)

    accuracy_test = accuracy_score(Y_test, Y_test_pred)

    mcc_test = matthews_corrcoef(Y_test, Y_test_pred)

    results.append({
        "Classifier": name,
        "Accuracy_Train": accuracy_train,

        "MCC_Train": mcc_train,
        "Accuracy_CV": accuracy_cv,

        "MCC_CV": mcc_cv,
        "Accuracy_Test": accuracy_test,

        "MCC_Test": mcc_test
    })

# Display the results
results_df = pd.DataFrame(results)
print(results_df)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import lightgbm as lgb

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Define regressor models and their hyperparameter grids
models = {
    'Decision Tree': (DecisionTreeRegressor(random_state=42), {'max_depth': [None, 5, 10, 20], 'min_samples_split': [2, 5, 10]}),
    'Extra Trees': (ExtraTreesRegressor(random_state=42), {'n_estimators': [100, 200, 500], 'max_depth': [None, 5, 10, 20]}),
    'Support Vector': (SVR(kernel='rbf', C=1), {'kernel': ['linear', 'rbf', 'poly'], 'C': [0.1, 1, 10]}),
    'k-Nearest Neighbors': (KNeighborsRegressor(), {'n_neighbors': [3, 5, 7, 9]}),
    'Random Forest': (RandomForestRegressor(random_state=42), {'n_estimators': [100, 200, 500], 'max_depth': [None, 5, 10, 20]}),
    'Gaussian Process': (GaussianProcessRegressor(), {'kernel': [1.0 * RBF(1.0)]}),
    'Gradient Boosting': (GradientBoostingRegressor(random_state=42), {'n_estimators': [100, 200, 500], 'learning_rate': [0.1, 0.05, 0.01]}),
    'LGBM': (lgb.LGBMRegressor(random_state=42), {'n_estimators': [100, 200, 500], 'learning_rate': [0.1, 0.05, 0.01]})

}
# Perform hyperparameter tuning for each model
results = {}
for model_name, (model, param_grid) in models.items():
    # Use GridSearchCV or RandomizedSearchCV based on your preference
    search = GridSearchCV(model, param_grid, cv=10, scoring='neg_mean_squared_error')
    search.fit(X_train, Y_train)

    best_model = search.best_estimator_
    best_params = search.best_params_

    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(Y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(Y_test, y_test_pred))

    train_r2 = r2_score(Y_train, y_train_pred)

    test_r2 = r2_score(Y_test, y_test_pred)

    # Calculate MAPE
    train_mape = mean_absolute_percentage_error(Y_train, y_train_pred)
    test_mape = mean_absolute_percentage_error(Y_test, y_test_pred)

    # Calculate Q2_CV using cross-validation
    cv_scores = cross_val_score(best_model, X_train, Y_train, cv=10, scoring='r2')
    Q2_CV = np.mean(cv_scores)

    results[model_name] = [best_params, {'train_rmse': train_rmse, 'train_r2': train_r2, 'test_rmse': test_rmse, 'test_r2': test_r2, 'Q2_CV': Q2_CV, 'train_mape': train_mape, 'test_mape': test_mape}]
    print(f"Model: {model_name}")
    print(f"Best Parameters: {best_params}")
    print(f"Train RMSE: {train_rmse:.4f}")
    print(f"Train R-squared: {train_r2:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    print(f"Test R-squared: {test_r2:.4f}")
    print(f"Q2_CV: {Q2_CV:.4f}")
    print(f"Train MAPE: {train_mape:.4f}")
    print(f"Test MAPE: {test_mape:.4f}")
    print()

# Compare results
best_model = min(results, key=lambda x: results[x]['test_rmse'])
print("Best Model:")
print(best_model)
print(results[best_model])

In [ ]:
train = pd.concat([pd.DataFrame(X_train), Y_train.reset_index(drop=True)], axis=1)
test = pd.concat([pd.DataFrame(X_test), Y_test.reset_index(drop=True)], axis=1)

train['model'] = "Train"
test['model'] = "Test"

In [ ]:
train['model'] = "Train"
test['model'] = "Test"

In [ ]:
frames = [train,test]
pca = pd.concat(frames)
pca.head(2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator
from pandas.plotting import scatter_matrix
# Plot the Figures Inline
%matplotlib inline


def PCA_plot(data):
    # PCA's components graphed in 2D
    # Apply Scaling
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    from matplotlib import pyplot as plt
    data_pca = data

    # Apply Scaling
    X = data_pca.drop('model', axis=1).values
    y = data_pca['model'].values

    # Formatting
    target_names = ['Train','Test']
    colors = ['blue','red']

    # 2 Components PCA

    fig=plt.figure(2, figsize=(7, 5))

    pca = PCA(n_components=2)
    X_std = StandardScaler().fit_transform(X)
    X_r = pca.fit_transform(X_std)

    for color, i, target_name in zip(colors, ['Train','Test'], target_names):
        plt.scatter(X_r[y == i, 0], X_r[y == i, 1],
                    color=color,

                    label=target_name)
    plt.xlabel('PC1',  fontstyle= "normal", fontsize=12, fontweight='bold')
    plt.ylabel('PC2',  fontstyle= "normal", fontsize=12, fontweight='bold')


    plt.grid(False) #remove grid in 2D plot

    # Add outline around axes
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(1)
        spine.set_color('black')

    # Legend with box and black outline
    legend = plt.legend(loc='best', scatterpoints=1)
    legend.get_frame().set_linewidth(2)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    plt.tick_params(direction="out", labelsize=12)
    plt.savefig('applicability_domain.pdf', dpi=1200)
PCA_plot(pca)

In [ ]:
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA

best_model = LGBMClassifier()
best_model.fit(X_train, Y_train)

# Applying PCA to training data
pca_train = PCA(n_components=2)
X_train_pca = pca_train.fit_transform(X_train)

# Applying PCA to test data
pca_test = PCA(n_components=2)
X_test_pca = pca_test.fit_transform(X_test)

# Visualize PCA of training data
plt.figure(figsize=(10, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], color='blue', label='Training-set')

plt.xlabel('PC1')
plt.ylabel('PC2')

# Overlay PCA of test data on the same plot
plt.scatter(X_test_pca[:, 0], X_test_pca[:, 1], color='red', label='Test-set')
plt.legend()

plt.show()
plt.savefig('PCA.pdf')


In [ ]:
import pandas as pd
df_final=pd.read_csv('/content/MMP13_pIC50_cleaned.csv')

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Assuming df_final is your DataFrame
features = ['MW', 'LogP', 'NumHDonors', 'NumHAcceptors', 'NumRotatableBonds', 'TPSA']

# Separate features and labels
X = df_final[features]
y = df_final['bioactivity_class']

# Standardize the data (important for PCA)
X_std = StandardScaler().fit_transform(X)

# Perform PCA
pca = PCA(n_components=2)  # You can change the number of components as needed
principal_components = pca.fit_transform(X_std)

# Create a new DataFrame with the principal components and group information
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['Groups'] = y.values

# Plot the PCA results
plt.figure(figsize=(6.5, 6.5))
colors = {'active': 'red', 'inactive': 'blue'}  # Adjust colors as needed
for group, color in colors.items():
    group_data = pca_df[pca_df['Groups'] == group]
    plt.scatter(group_data['PC1'], group_data['PC2'], c=color, label=group, alpha=0.25)


plt.xlabel('PC1', fontsize=14, fontweight='bold')
plt.ylabel('PC2', fontsize=14, fontweight='bold')
plt.legend()
plt.show()
plt.savefig('PCA of active.inactive.pdf')

In [ ]:
import pandas as pd
import scipy.stats

# Assuming 'df_final' is your DataFrame containing the relevant columns

# List of columns
columns_of_interest = ["MW", "LogP", "NumHAcceptors", "NumHDonors", "NumRotatableBonds", "TPSA"]

# Dictionary to store results
summary_statistics = {}

# Separate data for group1 and group2
for group_name in df_final['bioactivity_class'].unique():
    summary_df = df_final[df_final['bioactivity_class'] == group_name][columns_of_interest].describe().transpose()

    # Calculate skewness and kurtosis
    skewness_values = df_final[df_final['bioactivity_class'] == group_name][columns_of_interest].skew()
    kurtosis_values = df_final[df_final['bioactivity_class'] == group_name][columns_of_interest].kurt()

    # Add skewness and kurtosis to the summary DataFrame
    summary_df['skew'] = skewness_values
    summary_df['kurt'] = kurtosis_values

    # Add p-values to the table
    p_values = []
    for column in columns_of_interest:
        _, p_value = scipy.stats.ttest_ind(
            df_final[df_final['bioactivity_class'] == 'active'][column].dropna(),
            df_final[df_final['bioactivity_class'] == 'inactive'][column].dropna()
        )
        p_values.append(p_value)

    # Append p-values to the summary DataFrame
    summary_df['p-value'] = p_values

    # Store the summary DataFrame in the dictionary
    summary_statistics[group_name] = summary_df

# Display the results
for group_name, stats_df in summary_statistics.items():
    print(f"\nSummary statistics for {group_name}:")
    print(stats_df[['min', 'max', '50%', 'mean', 'skew', 'kurt', 'p-value']])